In [1]:
import onnx
from onnx import helper
from onnx import TensorProto
import numpy as np

In [2]:
in_dim = 5
out_dim = 5

X = helper.make_tensor_value_info(f"X", TensorProto.FLOAT, [1, in_dim])
Y = helper.make_tensor_value_info(f"Y", TensorProto.FLOAT, [1, out_dim])
rmean = helper.make_tensor(f"rmean", TensorProto.FLOAT, [in_dim],
                            np.random.rand(in_dim))
rvar = helper.make_tensor(f"rvar", TensorProto.FLOAT, [in_dim],
                            np.random.rand(in_dim))

scale = helper.make_tensor(f"scale", TensorProto.FLOAT, [in_dim],
                            np.random.rand(out_dim))
bias = helper.make_tensor(f"B", TensorProto.FLOAT, [in_dim],
                            np.random.rand(in_dim))

bn_node = helper.make_node("BatchNormalization",
                           ["X", "scale", "B", "rmean", "rvar"],
                           ["Y"],
                           name="onnx_bn")
grf = helper.make_graph([bn_node], "testgraph",
                        [X],
                        [Y],
                        [scale, bias, rmean, rvar])

opset = onnx.OperatorSetIdProto()
opset.version = 19
mdl = helper.make_model(grf, opset_imports = [opset])

onnx.checker.check_model(mdl)
onnx.shape_inference.infer_shapes(mdl, check_type=True, strict_mode=True, data_prop=True)

ir_version: 10
graph {
  node {
    input: "X"
    input: "scale"
    input: "B"
    input: "rmean"
    input: "rvar"
    output: "Y"
    name: "onnx_bn"
    op_type: "BatchNormalization"
  }
  name: "testgraph"
  initializer {
    dims: 5
    data_type: 1
    float_data: 0.403519511
    float_data: 0.849108338
    float_data: 0.604992449
    float_data: 0.468352437
    float_data: 0.891701639
    name: "scale"
  }
  initializer {
    dims: 5
    data_type: 1
    float_data: 0.436821193
    float_data: 0.183469296
    float_data: 0.467202842
    float_data: 0.872814715
    float_data: 0.314055055
    name: "B"
  }
  initializer {
    dims: 5
    data_type: 1
    float_data: 0.483134866
    float_data: 0.130410939
    float_data: 0.556482792
    float_data: 0.469330132
    float_data: 0.891790748
    name: "rmean"
  }
  initializer {
    dims: 5
    data_type: 1
    float_data: 0.246146366
    float_data: 0.90143317
    float_data: 0.789942503
    float_data: 0.560744822
    float_data:

In [3]:
from compiler import parsemodel, svimpl
spec = svimpl.FPGASpec(120, 600_000, 2_700_000, 100_000)
fpga_module = parsemodel.parse_model(mdl, in_dim, spec)
fpga_module.modules

[v_fifo_0:(1x5) [[0, wr_data_0, rd_data_0, wr_en_0, rd_en_0, v_fifo_0_out_vec_valid_0, 0]],
 vwb_mac_0:(1x5) [[in_data_ready, rd_data_0, wr_data_1, rd_en_0, wr_en_1, vwb_mac_0_out_vec_valid_0]],
 v_fifo_1:(1x5) [[0, wr_data_1, rd_data_1, wr_en_1, rd_en_1, v_fifo_1_out_vec_valid_1, 0]]]

In [4]:
len(fpga_module.modules[1].wfile.buffer.shape)

1

In [5]:
fpga_module.alloc_regs()
fpga_module.alloc_bram()
sv = fpga_module.make_sv()

In [6]:
fpga_module.modules[0].write_out_data.debug()

"{'name': 'rd_data_0', 'defined': True, 'num_elements': 5, 'n_bits': 8, 'tie_zero': False}"

In [7]:
import pprint
pprint.pprint(sv)

('\n'
 '\n'
 'logic [7:0] in_data_master;\n'
 'logic [4:0][7:0] rd_data_0;\n'
 'logic  wr_in_master;\n'
 'logic  rd_en_0;\n'
 'logic  v_fifo_0_out_vec_valid_0;\n'
 'logic  wrap_rd_0;\n'
 'v_fifo #(\n'
 '        .VecElements(5),\n'
 '        .ElementsPerRead(5),\n'
 '        .ElementsPerWrite(1),\n'
 '        .NBits(8)\n'
 '        .Depth(1))\n'
 '        v_fifo_0\n'
 '        (\n'
 '            .clk_in(clk_100mhz),\n'
 '            .rst_in(sys_rst),\n'
 '            .wr_en(wr_in_master),\n'
 '            .wr_data(in_data_master),\n'
 '            .rd_en(rd_en_0),\n'
 '            .rd_data(rd_data_0),\n'
 '            .wrap_rd(0)\n'
 '        );\n'
 '\n'
 '\n'
 'logic [7:0] wr_data_1;\n'
 'logic [4:0][7:0] out_data_master;\n'
 'logic  wr_en_1;\n'
 'logic  rd_out_master;\n'
 'logic  v_fifo_1_out_vec_valid_1;\n'
 'logic  wrap_rd_1;\n'
 'v_fifo #(\n'
 '        .VecElements(5),\n'
 '        .ElementsPerRead(5),\n'
 '        .ElementsPerWrite(5),\n'
 '        .NBits(8)\n'
 '        .Depth(1)

In [8]:
with open("dummy_model.sv", 'w') as f:
    f.write(sv)